In [ ]:

import json
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from pathlib import Path


In [ ]:

INPUT_HOURS = 48
HORIZON = 24
FEATURES = ["temp_c", "humidity", "wind_kmh", "precip_prob"]
TARGET = "temp_c"


In [ ]:

df = pd.read_csv("../data/weather_kriviyrih.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.sort_values("timestamp").reset_index(drop=True)


In [ ]:

scaler = StandardScaler()
X_all = scaler.fit_transform(df[FEATURES])
y_all = df[TARGET].values


In [ ]:

X, y = [], []
for i in range(len(df) - INPUT_HOURS - HORIZON + 1):
    X.append(X_all[i:i+INPUT_HOURS])
    y.append(y_all[i+INPUT_HOURS:i+INPUT_HOURS+HORIZON])
X = np.array(X, dtype="float32")
y = np.array(y, dtype="float32")
X.shape, y.shape


In [ ]:

split = int(len(X) * 0.8)
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]


In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(INPUT_HOURS, len(FEATURES))),
    tf.keras.layers.Conv1D(64, 5, padding="same", activation="relu"),
    tf.keras.layers.Conv1D(64, 5, padding="same", activation="relu"),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(HORIZON)
])
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()


In [ ]:

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True)]
)


In [ ]:

Path("../artifacts").mkdir(exist_ok=True)
model.save("../artifacts/keras_model.keras")
scaler_json = {
    "mean": scaler.mean_.tolist(),
    "scale": scaler.scale_.tolist(),
    "features": FEATURES,
    "input_hours": INPUT_HOURS,
    "horizon": HORIZON
}
with open("../artifacts/scaler.json", "w", encoding="utf-8") as f:
    json.dump(scaler_json, f, indent=2)
print("Model & scaler saved")
